
<br>
Observation des relations entre les différentes occupations des écrivain-es.<br>


In [1]:
import sparql_dataframe
import pandas
import re

L'adresse de DBPedia, où la requête doit être adressée.

In [2]:
dbpedia = "http://dbpedia.org/sparql"

Les requêtes SPARQL. Je sépare en plusieurs requêtes afin de pouvoir avoir plus que 10'000 résultats. Les requêtes portent sur les occupations des personnes ayant une des occupations suivantes: Novelist, Poet, Dramatist, Writers.

In [3]:
queries = {}
dataframe = {}
writing_group = ["Writer", "Novelist", "Poet", "Playwright"]
for i in writing_group:
    queries[i] = (
        "PREFIX dbo: <http://dbpedia.org/ontology/>\n"
        "PREFIX dbr: <http://dbpedia.org/resource/>\n"
        "\n"
        "SELECT DISTINCT ?person ?occupation\n"
        "WHERE {\n"
        f"    ?person ?a dbr:{i} ;\n"
        "            dbo:occupation ?occupation .\n"
        "}\n"
    )
    dataframe[i] = sparql_dataframe.get(dbpedia, queries[i])
    dataframe[i]["Group"] = i
    print(i, ":", len(dataframe[i]))

Writer : 10000
Novelist : 5976
Poet : 8452
Playwright : 5021


Assembler (concaténer) les dataframes en un seul dataframe.

In [4]:
df = pandas.concat(dataframe.values())
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29449 entries, 0 to 5020
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   person      29449 non-null  object
 1   occupation  29449 non-null  object
 2   Group       29449 non-null  object
dtypes: object(3)
memory usage: 920.3+ KB


Je simplifie les URIs en noms (en enlevant le début).

In [5]:
df.replace(
    r"http://dbpedia\.org/(?:resource|ontology)/(.*)",
    "\\1",
    regex=True,
    inplace=True,
)
len(df)

29449

Comme on peut le voir en observant le début du tableau, une ligne ne correspond pas à une personne, mais à une occupation. Puisque ce qui va m'intéresser ici consiste dans la multiplication des activités, (par exemple poet + translater + musician) je ne vais chercher à réduire ces lignes à des personnes, et travaillerai sur des relations occupation-occupation.

In [6]:
df.head()

,person,occupation,Group
0,Caiseal_Mór,Musician,Writer
1,Caiseal_Mór,Artist,Writer
2,Caiseal_Mór,Writer,Writer
3,Caitlin_Flanagan,Caitlin_Flanagan__PersonFunction__1,Writer
4,Caitlin_Flanagan,Writer,Writer


Le tableau contient de nombreuses lignes dont la valeur de la colonne "occupation" est en fait le nom de la personne associé à la mention "PersonFunction".

In [7]:
df[df["occupation"].str.contains("PersonFunction")]

,person,occupation,Group
3,Caitlin_Flanagan,Caitlin_Flanagan__PersonFunction__1,Writer
10,Carl_Washington,Carl_Washington__PersonFunction__1,Writer
18,Carlo_Mattogno,Carlo_Mattogno__PersonFunction__1,Writer
21,Carlos_Alberto_Montaner,Carlos_Alberto_Montaner__PersonFunction__1,Writer
25,Carlos_Francisco_Chang_Marín,Carlos_Francisco_Chang_Marín__PersonFunction__1,Writer
...,...,...,...
4988,Ted_Hamilton__OAM__1,Ted_Hamilton__OAM__1__PersonFunction__1,Playwright
4999,Andrew_Nicholls_and_Darrell_Vickers__Darrell_V...,Andrew_Nicholls_and_Darrell_Vickers__Darrell_V...,Playwright
5004,Leah_Purcell__Leah_Purcell__1,Leah_Purcell__Leah_Purcell__1__PersonFunction__1,Playwright
5008,Mayo_Simon__Mayo_Simon__1,Mayo_Simon__Mayo_Simon__1__PersonFunction__1,Playwright


Ces lignes ne sont pas utiles pour moi. Je les retire donc.

In [8]:
df = df[~df["occupation"].str.contains("PersonFunction")]
len(df)

21325

Les nombres d'occupations est très elevé.

In [9]:
len(set(df.occupation))

1645

Et certaines occupations identiques apparaissent sous des noms différents.

In [10]:
[i for i in set(df.occupation) if i.startswith("Drama")]

['Drama',
 'Drama_teacher',
 'Dramaturg',
 'Dramaturgy',
 'Dramatist',
 'Drama_critic',
 'Dramaturge']

Il faut donc procéder à des regroupements. Un tri manuel serait beaucoup trop laborieux, et poserait problème pour répéter l'opération si des nouveaux résultats venaient s'ajouter. Je vais donc opter plutôt pour une approche plus approximative mais permettant d'automatiser cette opération et de l'appliquer à des données en nombre important. L'idée est d'utiliser, comme dans l'exemple ci-dessus, la correspondance de motifs pour sélectionner et grouper des occupations. On peut par exemple grouper un certains nombres d'occupations qui se terminent en "gist", et qui, généralement, désignent des activités intellectuelles spécialisées. Cette manière de faire est approximative et les résultats obtenus contiennent des erreurs (ex. Suffragist, Collagist), mais ça permet de travailler rapidement et avec une certaine souplesse dans l'éventualité où les données changeraient.

In [11]:
[i for i in set(df.occupation) if re.search("gist$", i)]

['Musicologist',
 'Numerologist',
 'Philologist',
 'Criminologist',
 'Life_strategist',
 'Archeologist',
 'Dermatologist',
 'Biologist',
 'Parapsychologist',
 'Theatrologist',
 'Neurologist',
 'Phrenologist',
 'Seismologist',
 'Psychologist',
 'Geologist',
 'Collagist',
 'Anthologist',
 'Ufologist',
 'Gynecologist',
 'Suffragist',
 'Sociologist',
 'Culturologist',
 'Zoologist',
 'Meteorologist',
 'Marine_biologist',
 'Sexologist',
 'Archaeologist',
 'Monologist',
 'Ethnologist',
 'Etymologist',
 'Pharmacologist',
 'Cultural_anthropologist',
 'Museologist',
 'Speleologist',
 'Genealogist',
 'Iranologist',
 'Turkologist',
 'Political_strategist',
 'Anti-suffragist',
 'Forensic_anthropologist',
 'Clinical_psychologist',
 'Indologist',
 'Anthropologist']

Pour la suite de l'exploration de ces données, l'objectif est d'observer les différences dans les activités annexes à l'activité de création littéraire entre les sous-groupes que constituent les "poets", les "dramatists" et les "novelists" qui constituent les trois genres dominants de la production littéraire occidentale moderne. Je n'intègre pas l'essai ni l'autobiographie, qui représentent des cas à part et sont difficiles à distinguer de pratiques non-littéraires en raison de leur caractère non-fictionnel (ex. les autobiographie de star, ou les essais de développement personnel). L'idée est de voir si certaines activités annexes sont sur- ou sous-représentrées dans certains de ces sous-groupes. Par exemple, trouve-t-on davantages de personnes exerçant des activités religieuses ou mystiques chez les poète-sses que chez les dramaturges? À l'inverse, les dramaturges ont-iels en revanche plus tendance que les poète-sses à écrire pour le cinéma, et les romancier-ères pour la presse écrite? Les poète-sses font-iels plus de musique, les romancier-ères plus de peinture?

In [12]:
occupations_grouped = {
    "intellectual": [
        "gist",
        "stud",
        "scho",
        "critic",
        "prof",
        "lectur",
        "univ",
        "search",
        "anthro",
        "socio",
        "ethno",
        "hist",
        "theor",
        "lingu",
        "philo",
        "chemis",
        "biol",
        "math",
        "scienc",
        "lexico",
    ],
    "social sciences": ["socio", "anthro", "ethno", "hist"],
    "natural sciences": ["biol", "math", "phys", "chemis"],
    "non-written media": [
        "radio",
        "screen",
        "cinem",
        "script",
        "film",
    ],
    "press": ["journ", "report", "magaz", "news", "chron"],
    "religion": [
        "theol",
        "relig",
        "priest",
        "preach",
        "saint",
        "church",
        "myst",
    ],
    "visual arts": [
        "comic",
        "paint",
        "photo",
        "illust",
        "draw",
        "video",
    ],
    "music": ["music", "compos"],
    "books": ["edit", "book", "publish", "print", "libr"],
    "politic": [
        "politic",
        "deput",
        "govern",
        "presid",
        "law",
        "lega",
        "juri",
    ],
    "education": ["school", "teach", "pedag", "educ"],
    "translation": ["transl"],
}

Je mets en caractères minuscules les occupations, afin de pouvoir effectuer plus facilement les comparaison avec les mots déterminés ci-dessus. Je construis une nouvelle liste à double éléments: l'occupation et le "writing_group" (Novelist, Poet, Dramatist, Writer).

In [13]:
occupation_and_group = [(i[1].lower(), i[2]) for i in df.values]
occupation_and_group[:10]

[('musician', 'Writer'),
 ('artist', 'Writer'),
 ('writer', 'Writer'),
 ('writer', 'Writer'),
 ('social_critic', 'Writer'),
 ('novelist', 'Writer'),
 ('actor', 'Writer'),
 ('writer', 'Writer'),
 ('film_producer', 'Writer'),
 ('psychiatrist', 'Writer')]

À travers un enchâssement de boucles, je compte, pour chaque "writing_group", combien de lignes correspondent à chacun de ces groupes d'occupations (politic, education, etc.).

In [14]:
count = {}
for i in writing_group:
    count[i] = {}
    for word_group in occupations_grouped:
        count[i][word_group] = 0
        for row in [r for r in occupation_and_group if r[1] == i]:
            for word in occupations_grouped[word_group]:
                if word in row[0]:
                    count[i][word_group] = count[i][word_group] + 1
                    break
count

{'Writer': {'intellectual': 482,
  'social sciences': 113,
  'natural sciences': 29,
  'non-written media': 677,
  'press': 332,
  'religion': 20,
  'visual arts': 118,
  'music': 111,
  'books': 155,
  'politic': 127,
  'education': 76,
  'translation': 72},
 'Novelist': {'intellectual': 249,
  'social sciences': 45,
  'natural sciences': 20,
  'non-written media': 342,
  'press': 241,
  'religion': 10,
  'visual arts': 28,
  'music': 26,
  'books': 93,
  'politic': 58,
  'education': 39,
  'translation': 55},
 'Poet': {'intellectual': 488,
  'social sciences': 70,
  'natural sciences': 35,
  'non-written media': 134,
  'press': 204,
  'religion': 22,
  'visual arts': 62,
  'music': 103,
  'books': 116,
  'politic': 117,
  'education': 100,
  'translation': 203},
 'Playwright': {'intellectual': 128,
  'social sciences': 20,
  'natural sciences': 6,
  'non-written media': 457,
  'press': 88,
  'religion': 2,
  'visual arts': 15,
  'music': 59,
  'books': 81,
  'politic': 34,
  'educati

Les données peuvent être remises dans un DataFrame pandas:

In [15]:
occount = pandas.DataFrame(count)
occount

,Writer,Novelist,Poet,Playwright
intellectual,482,249,488,128
social sciences,113,45,70,20
natural sciences,29,20,35,6
non-written media,677,342,134,457
press,332,241,204,88
religion,20,10,22,2
visual arts,118,28,62,15
music,111,26,103,59
books,155,93,116,81
politic,127,58,117,34


Un rapide regard des données donne déjà des informations intéressantes. Par exemple, l'activité de traduction, qui est une activité importante des poète-sses modernistes et contemporain-es, apparaît nettement comme une spécificité de ce groupe. Ce n'est pas une surprise, pour plusieurs raisons: (1) la traduction de poésie s'accomplit plus rapidement et peut donc cohabiter plus facilement avec une activité d'écriture; (2) la traduction de la poésie pose des problèmes complexes qui la font souvent percevoir comme une forme de création à part entière et au premier degré, et de fait il y a des traductions célèbres de poésie (ex. Poe traduit par Baudelaire); (3) éventuellement, la recherche d'innovation linguistique, peut-être plus présente en poésie que dans l'écriture romanesque, pousserait davantage les poéte-sses à lire ce qui se fait ailleurs -- la recherche d'un écart avec le langage ordinaire (qui caractérise la poésie) peut également être une source de motivation à lire et faire circuler des textes en langue étrangère. Il s'agit également du seul groupe chez lequel le groupe d'occupation "press" est plus élevé que le groupe "non-written media". Ici encore, ce n'est pas surprenant: la poésie étant, contrairement aux écritures romanesque et dramatique, souvent non-narrative, les poète-sses ont naturellement moins de relation avec le cinéma (du moins le cinéma traditionnel, où leurs compétences s'exportent plus difficilement). Par ailleurs, peut-être les groupes ne sont-ils pas distribués de façon homogène dans le temps: il est possible que si les poète-sses ont moins travaillé pour la télévision, c'est car la plupart des poète-sses de mon jeu de données sont né-es et mort-es avant l'essor de la télévision.

Ajouter une colonne "total" à droite afin de pouvoir plutôt travailler avec des nombres relatifs aux totaux.

In [16]:
occount["total"] = occount.sum(axis=1)
occount.loc["total"] = occount.sum(numeric_only=True, axis=0)
occount

,Writer,Novelist,Poet,Playwright,total
intellectual,482,249,488,128,1347
social sciences,113,45,70,20,248
natural sciences,29,20,35,6,90
non-written media,677,342,134,457,1610
press,332,241,204,88,865
religion,20,10,22,2,54
visual arts,118,28,62,15,223
music,111,26,103,59,299
books,155,93,116,81,445
politic,127,58,117,34,336


In [17]:
print("proportion de traduction:\n")
for i in writing_group:
    print(i, ":", round(occount[i].translation / occount[i].total, 2))

proportion de traduction:

Writer : 0.03
Novelist : 0.05
Poet : 0.12
Playwright : 0.05


La proportion d'occupation-traduction chez les poète-sses est largement supérieure à celles qu'on trouve chez les romancier-ères et dramaturges, lesquelles sont identiques (0.05). Le rapport entre la proportion chez les poète-sses et chez les dramaturges, de 1/2, est identique au rapport entre dramaturge et writers, donc il pourrait sembler peu significatif. Mais cela n'est à mon avis pas le cas. Car la proportion plus basse chez les "Writers" s'explique autrement: en effet, ce groupe est constitué d'un nombre important d'auteurices identifié comme "writer" pour des activités non-littéraires (ex. écrivain-es de livres scientifiques, de philosophie, etc.), et qui n'ont pas la situation économique précaire des auteurices littéraires, puisque les Writers sont aussi des neurologistes reconnu-es publiant des essais de vulgarisation, etc. Il va de soit que l'activité de traduction dans ces catégorie socioprofessionnelle est une activité très secondaire; puisque l'écriture ne constitue pas nécessairement leur compétence principale, les individus de ce groupe n'ont aucune raison de la mobiliser pour en faire une activité rémunératrice.

Calculer l'effectif attendu des poète-sses traducteurices.

In [18]:
occount.total.translation * (occount.Poet.total / occount.total.total)

101.54014004233838

Le rapport entre effectif attendu et effectif observé.

In [19]:
round(
    occount.Poet.translation
    / (
        occount.total.translation
        * (occount.Poet.total / occount.total.total)
    ),
    2,
)

2.0

Vérifier, enfin, que le nombre élevé de poète-sses traducteurices n'est pas lié à un biais dans ma méthode de groupement des occupations. Puisque les deux seuls résultats sont "Translation" et "Translator", il n'y a pas de biais à ce niveau-là.

In [20]:
set([
    i
    for i in list(dataframe["Poet"].occupation)
    if re.search("[Tt]ransl", i) and 'PersonFunction' not in i
])

{'http://dbpedia.org/resource/Translation',
 'http://dbpedia.org/resource/Translator'}